## 0. Libraries import

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# imports 
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
#train test split
from sklearn.model_selection import train_test_split
# Hyperparameters selection
from sklearn.model_selection import RandomizedSearchCV
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
# models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
# error
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

## 1. Data import

### 1.1 Impor train data

In [3]:
df_diamonds_train=pd.read_csv('../data/diamonds_train.csv')
df_diamonds_train.pop("Unnamed: 0") #dropped an unnecessary column
df_diamonds_train

,index_id,depth,table,x,y,z,price,carat,cut,color,clarity,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,505,0.32,Very Good,H,VS2,Kimberly
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,2686,0.71,Fair,G,VS1,Las Vegas
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,738,0.41,Good,D,SI1,Kimberly
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,4882,1.02,Ideal,G,SI1,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43,10070,1.34,Ideal,G,VS1,Antwerp
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73,12615,2.02,Good,F,SI2,Madrid
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01,5457,1.01,Ideal,H,SI1,Kimberly
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76,456,0.33,Ideal,J,VS1,Kimberly


### 1.2 Impor test data

In [4]:
df_diamonds_test=pd.read_csv('../data/diamonds_test.csv')
df_diamonds_test

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


## 2. Data preparation (training set)

### 2.1 Defining numerical and categorical features

In [5]:
# Excluding city (with trees sometimes is better to keep variables even if they are correlated)
num_features_list=['x','y','z','depth','table','carat']
cat_features_list=['cut','color','clarity','city']
target='price'
features_list=['x','y','z','depth','table','carat','cut','color','clarity','city']

### 3.5 One hot encoding

In [6]:
# 3. Defining features y target
#features_list_volume=['x', 'y', 'z', 'depth', 'table', 'carat', 'cut', 'color', 'clarity','volume']
X=df_diamonds_train[features_list]
y=df_diamonds_train['price']

In [7]:
for column in cat_features_list:
    X[column]=X[column].astype('category')
    X[column]=X[column].cat.codes

In [8]:
X

,x,y,z,depth,table,carat,cut,color,clarity,city
0,6.83,6.79,4.25,62.4,58.0,1.21,3,6,5,2
1,4.35,4.38,2.75,63.0,57.0,0.32,4,4,5,3
2,5.62,5.53,3.65,65.5,55.0,0.71,0,3,4,4
3,4.68,4.72,3.00,63.8,56.0,0.41,1,0,2,3
4,6.55,6.51,3.95,60.5,59.0,1.02,2,3,2,2
...,...,...,...,...,...,...,...,...,...,...
40450,7.10,7.04,4.43,62.7,57.0,1.34,2,3,4,1
40451,8.31,8.25,4.73,57.1,60.0,2.02,1,2,3,7
40452,6.37,6.42,4.01,62.7,56.0,1.01,2,4,2,3
40453,4.45,4.47,2.76,61.9,54.3,0.33,2,6,4,3


### 3.7 Define train and validation

In [9]:
# 5.Splitting train and test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(32364, 10)

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32364 entries, 32121 to 15795
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   x        32364 non-null  float64
 1   y        32364 non-null  float64
 2   z        32364 non-null  float64
 3   depth    32364 non-null  float64
 4   table    32364 non-null  float64
 5   carat    32364 non-null  float64
 6   cut      32364 non-null  int8   
 7   color    32364 non-null  int8   
 8   clarity  32364 non-null  int8   
 9   city     32364 non-null  int8   
dtypes: float64(6), int8(4)
memory usage: 1.9 MB


## 4. Data preparation (test set)

## 5. Feature engineering (test set) 

In [12]:
# 0. Adapting categorical features for validation model
X_test=df_diamonds_test[features_list]

### 3.6 Change categorical variables to code

In [13]:
for column in cat_features_list:
    X_test[column]=X_test[column].astype('category')
    X_test[column]=X_test[column].cat.codes

In [14]:
X_test

,x,y,z,depth,table,carat,cut,color,clarity,city
0,5.82,5.89,3.67,62.7,60.0,0.79,4,2,2,0
1,6.81,6.89,4.18,61.0,57.0,1.20,2,6,4,10
2,7.38,7.32,4.57,62.2,61.0,1.57,3,4,2,3
3,6.09,6.13,3.90,63.8,54.0,0.90,4,2,2,3
4,5.05,5.09,3.19,62.9,58.0,0.50,4,2,4,0
...,...,...,...,...,...,...,...,...,...,...
13480,5.35,5.32,3.30,61.9,56.0,0.57,2,1,2,0
13481,5.71,5.73,3.56,62.2,55.0,0.71,2,5,5,8
13482,5.75,5.71,3.53,61.6,55.0,0.70,2,2,4,11
13483,5.85,5.89,3.45,58.8,57.0,0.70,4,2,3,10


In [15]:
X_test.shape

(13485, 10)

In [16]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13485 entries, 0 to 13484
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   x        13485 non-null  float64
 1   y        13485 non-null  float64
 2   z        13485 non-null  float64
 3   depth    13485 non-null  float64
 4   table    13485 non-null  float64
 5   carat    13485 non-null  float64
 6   cut      13485 non-null  int8   
 7   color    13485 non-null  int8   
 8   clarity  13485 non-null  int8   
 9   city     13485 non-null  int8   
dtypes: float64(6), int8(4)
memory usage: 684.9 KB


## Model definition - RandomForestRegressor - with Random Hyperparameter Grid

RandomForestRegressor: multiple trees in paralel changing samples and convining diferrent features (overfitting when the tree is big and good to reduce error variance)

Main Parameters:
   - bootstrap -> method for sampling data points (TRUE bagging and FALSE pasting, with/without replacement)
   - n_estimators -> number of trees in the foreset
   - max_depth -> max number of levels in each decision tree
   - max_features -> max number of features considered for splitting a node
   - ccp_alpha ->
   - criterion ->
   - max_leaf_nodes -> max number of solution nodes 
   - max_samples ->
   - min_impurity_decrease ->
   - min_samples_leaf -> min number of data points allowed in a leaf node
   - min_samples_split -> min number of data points placed in a node before the node is split
   - min_weight_fraction_leaf
   - n_estimators -> number of trees in the foreset
   - n_jobs 
   - oob_score 
   - random_state
   - verbose
   - warm_start   

In [17]:
# 1. XGBRegressor 
model = XGBRegressor(n_estimators=200,colsample_bylevel=1,colsample_bynode=1,
                     colsample_bytree=0.8,reg_alpha=1, reg_lambda=1,gamma=0,learning_rate=0.1, random_state=42)
hyperparameters = model.get_params()
print(type(model), '\n')

<class 'xgboost.sklearn.XGBRegressor'> 



## Model training with validation

In [19]:
%%time
# 1. XGBRegressor early_stopping_rounds=40
# Model training
model.fit(X_train, y_train,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5050.90381	validation_1-rmse:5089.29590
[1]	validation_0-rmse:4563.36816	validation_1-rmse:4601.79346
[2]	validation_0-rmse:4125.61230	validation_1-rmse:4162.69678
[3]	validation_0-rmse:3731.59399	validation_1-rmse:3769.20801
[4]	validation_0-rmse:3382.80981	validation_1-rmse:3418.59229
[5]	validation_0-rmse:3065.16113	validation_1-rmse:3099.92847
[6]	validation_0-rmse:2785.76831	validation_1-rmse:2818.56006
[7]	validation_0-rmse:2529.17700	validation_1-rmse:2560.83228
[8]	validation_0-rmse:2299.06958	validation_1-rmse:2329.96802
[9]	validation_0-rmse:2099.20337	validation_1-rmse:2129.87061
[10]	validation_0-rmse:1921.32410	validation_1-rmse:1952.65137
[11]	validation_0-rmse:1763.24255	validation_1-rmse:1795.07227
[12]	validation_0-rmse:1622.28272	validation_1-rmse:1654.87793
[13]	validation_0-rmse:1491.03357	validation_1-rmse:1523.75659
[14]	validation_0-rmse:1380.28589	validation_1-rmse:1413.98059
[15]	validation_0-rmse:1273.08154	validation_1-rmse:1307.58948
[1

[134]	validation_0-rmse:420.50696	validation_1-rmse:539.32593
[135]	validation_0-rmse:419.11578	validation_1-rmse:539.53381
[136]	validation_0-rmse:418.88226	validation_1-rmse:539.55164
[137]	validation_0-rmse:417.59528	validation_1-rmse:539.97266
[138]	validation_0-rmse:416.97806	validation_1-rmse:539.77026
[139]	validation_0-rmse:415.62097	validation_1-rmse:539.71728
[140]	validation_0-rmse:415.15204	validation_1-rmse:539.30841
[141]	validation_0-rmse:414.77664	validation_1-rmse:539.03082
[142]	validation_0-rmse:414.48874	validation_1-rmse:538.92523
[143]	validation_0-rmse:413.84265	validation_1-rmse:538.71954
[144]	validation_0-rmse:413.60144	validation_1-rmse:538.84168
[145]	validation_0-rmse:412.85437	validation_1-rmse:538.55835
[146]	validation_0-rmse:411.77054	validation_1-rmse:538.66882
[147]	validation_0-rmse:410.83402	validation_1-rmse:538.78094
[148]	validation_0-rmse:410.68701	validation_1-rmse:538.83575
[149]	validation_0-rmse:410.51224	validation_1-rmse:538.85663
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
%%time
# Model predictions
y_pred_val = model.predict(X_val)

CPU times: user 82.7 ms, sys: 4.98 ms, total: 87.7 ms
Wall time: 18.9 ms


In [21]:
%%time
# Model predictions
y_pred_train = model.predict(X_train)

CPU times: user 233 ms, sys: 5.55 ms, total: 239 ms
Wall time: 39.8 ms


## Training set error

In [22]:
# Model predictions
rmse_train = mean_squared_error(y_train, y_pred_train)**0.5
rmse_train

389.74167410208605

In [23]:
mae_train=mean_absolute_error(y_train, y_pred_train)
mae_train

221.67890001183136

In [24]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9822316014921281

## Model validation

In [25]:
#432
rmse_val = mean_squared_error(y_val, y_pred_val)**0.5
rmse_val

537.917038299929

In [26]:
mae_val=mean_absolute_error(y_val, y_pred_val)
mae_val

275.0441958651733

In [27]:
r2r = r2_score(y_val, y_pred_val)
r2r

0.9822316014921281

## Model training without validation

In [34]:
%%time
# Model training
model.fit(X, y,eval_set=[(X_train,y_train),(X_val,y_val)],early_stopping_rounds=40)

[0]	validation_0-rmse:5050.68359	validation_1-rmse:5087.80469
[1]	validation_0-rmse:4562.20605	validation_1-rmse:4599.24902
[2]	validation_0-rmse:4123.91309	validation_1-rmse:4159.14795
[3]	validation_0-rmse:3729.33838	validation_1-rmse:3765.17456
[4]	validation_0-rmse:3380.39526	validation_1-rmse:3414.10815
[5]	validation_0-rmse:3062.67505	validation_1-rmse:3095.13867
[6]	validation_0-rmse:2781.86133	validation_1-rmse:2812.09204
[7]	validation_0-rmse:2526.08789	validation_1-rmse:2556.05298
[8]	validation_0-rmse:2296.61279	validation_1-rmse:2325.08301
[9]	validation_0-rmse:2096.46216	validation_1-rmse:2125.00977
[10]	validation_0-rmse:1918.47144	validation_1-rmse:1946.22485
[11]	validation_0-rmse:1760.03711	validation_1-rmse:1786.45325
[12]	validation_0-rmse:1619.28235	validation_1-rmse:1645.33813
[13]	validation_0-rmse:1486.47119	validation_1-rmse:1511.07886
[14]	validation_0-rmse:1376.82519	validation_1-rmse:1400.11145
[15]	validation_0-rmse:1271.31812	validation_1-rmse:1293.66028
[1

[134]	validation_0-rmse:434.29211	validation_1-rmse:432.44083
[135]	validation_0-rmse:433.94797	validation_1-rmse:432.17407
[136]	validation_0-rmse:433.75366	validation_1-rmse:432.03824
[137]	validation_0-rmse:432.70059	validation_1-rmse:431.13834
[138]	validation_0-rmse:432.62927	validation_1-rmse:431.06696
[139]	validation_0-rmse:431.88379	validation_1-rmse:430.20017
[140]	validation_0-rmse:431.47238	validation_1-rmse:429.81763
[141]	validation_0-rmse:430.55997	validation_1-rmse:428.77991
[142]	validation_0-rmse:430.26144	validation_1-rmse:428.54108
[143]	validation_0-rmse:429.70792	validation_1-rmse:427.84976
[144]	validation_0-rmse:429.05954	validation_1-rmse:427.35574
[145]	validation_0-rmse:427.98053	validation_1-rmse:426.27838
[146]	validation_0-rmse:426.72931	validation_1-rmse:424.81601
[147]	validation_0-rmse:426.63007	validation_1-rmse:424.72739
[148]	validation_0-rmse:426.40491	validation_1-rmse:424.56296
[149]	validation_0-rmse:425.83975	validation_1-rmse:424.16827
[150]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Test Preditions

In [35]:
predictions = model.predict(X_test)

In [36]:
predictions=pd.DataFrame(predictions)

In [37]:
predictions.reset_index(inplace=True)

In [38]:
predictions=predictions.rename({0: 'price','index': 'id'}, axis=1)

## Save Preditions

In [39]:
predictions.to_csv('../data/diamonds_predictions_catcodes_XGB.csv',index=False)